In [143]:
from copy import deepcopy
from pathlib import Path
import pathlib

import numpy as np
import pandas as pd
from scipy.special import factorial

def neuropy_bayesian_prob(tau, P_x, F, n, use_flat_computation_mode=True, debug_print=False):
    """ 
        n_i: the number of spikes fired by each cell during the time window of consideration
        use_flat_computation_mode: bool - if True, a more memory efficient accumulating computation is performed that avoids `MemoryError: Unable to allocate 65.4 GiB for an array with shape (3969, 21896, 101) and data type float64` caused by allocating the full `cell_prob` matrix
            ERROR: it looks like this `use_flat_computation_mode` broke things and the outputs have been all-NaNs ever since?


    NOTES: Flat vs. Full computation modes:
    Originally 
        cell_prob = np.zeros((nFlatPositionBins, nTimeBins, nCells)) 
    This was updated throughout the loop, and then after the loop completed np.prod(cell_prob, axis=2) was used to collapse along axis=2 (nCells), leaving the output posterior with dimensions (nFlatPositionBins, nTimeBins)

    To get around this, I introduced a version that accumulates the multilications over the course of the loop.
        cell_prob = np.zeros((nFlatPositionBins, nTimeBins))

    """
    assert(len(n) == np.shape(F)[1]), f'n must be a column vector with an entry for each place cell (neuron). Instead it is of np.shape(n): {np.shape(n)}. np.shape(F): {np.shape(F)}'        
    if debug_print:
        print(f'np.shape(P_x): {np.shape(P_x)}, np.shape(F): {np.shape(F)}, np.shape(n): {np.shape(n)}')
    # np.shape(P_x): (1066, 1), np.shape(F): (1066, 66), np.shape(n): (66, 3530)
    
    nCells = n.shape[0]
    nTimeBins = n.shape[1] # many time_bins
    nFlatPositionBins = np.shape(P_x)[0]

    F = F.T # Transpose F so it's of the right form
    n = n.astype(F.dtype) # convert the data type of n
    
    if use_flat_computation_mode:
        ## Single-cell flat version which updates each iteration:
        # cell_prob = np.zeros((nFlatPositionBins, nTimeBins), dtype=F.dtype) ## MemoryError: Unable to allocate 65.4 GiB for an array with shape (3969, 21896, 101) and data type float64
        cum_cell_prob = np.ones((nFlatPositionBins, nTimeBins), dtype=F.dtype)
        
    else:
        # Full Version which leads to MemoryError when nCells is too large:
        cell_prob = np.zeros((nFlatPositionBins, nTimeBins, nCells)) ## MemoryError: Unable to allocate 65.4 GiB for an array with shape (3969, 21896, 101) and data type float64

    for cell in np.arange(nCells):
        """ Comparing to the Zhang paper: the output posterior is P_n_given_x (Eqn 35)
            cell_ratemap: [f_{i}(x) for i in range(nCells)]
            cell_spkcnt: [n_{i} for i in range(nCells)]            
        """
        cell_spkcnt = n[cell, :][np.newaxis, :] # .shape: (1, nTimeBins)
        cell_ratemap = F[cell, :][:, np.newaxis] # .shape: (nFlatPositionBins, 1)
        coeff = 1.0 / (factorial(cell_spkcnt)) # 1/factorial(n_{i}) term # .shape: (1, nTimeBins)
        

        if use_flat_computation_mode:
            # Single-cell flat Version:
            cum_cell_prob *= (((tau * cell_ratemap) ** cell_spkcnt) * coeff) * (np.exp(-tau * cell_ratemap)) # product equal using *=  # .shape (nFlatPositionBins, nTimeBins)
        else:
            # Full Version:
            # broadcasting
            cell_prob[:, :, cell] = (((tau * cell_ratemap) ** cell_spkcnt) * coeff) * (
                np.exp(-tau * cell_ratemap)
            )

    if use_flat_computation_mode:
        # Single-cell flat Version:
        posterior = cum_cell_prob # The product has already been accumulating all along
        posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n
    else:
        # Full Version:
        posterior = np.prod(cell_prob, axis=2) # note this product removes axis=2 (nCells)
        posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n

    return posterior


def neuropy_bayesian_prob_flat_only(tau, P_x, F, n, debug_print=False):
    """   n_i: the number of spikes fired by each cell during the time window of consideration
    """
    assert(len(n) == np.shape(F)[1]), f'n must be a column vector with an entry for each place cell (neuron). Instead it is of np.shape(n): {np.shape(n)}. np.shape(F): {np.shape(F)}'        
    if debug_print:
        print(f'np.shape(P_x): {np.shape(P_x)}, np.shape(F): {np.shape(F)}, np.shape(n): {np.shape(n)}')
    # np.shape(P_x): (1066, 1), np.shape(F): (1066, 66), np.shape(n): (66, 3530)
        
    ## Fixed version:
    nCells = n.shape[0]
    nTimeBins = n.shape[1] # many time_bins
    nFlatPositionBins = np.shape(P_x)[0]
    F = F.T # Transpose F so it's of the right form
    n = n.astype(F.dtype)

    ## Single-cell flat version which updates each iteration:
    cum_cell_prob = np.ones((nFlatPositionBins, nTimeBins), dtype=F.dtype)
    for cell in np.arange(nCells):
        """ Comparing to the Zhang paper: the output posterior is P_n_given_x (Eqn 35)
            cell_ratemap: [f_{i}(x) for i in range(nCells)]
            cell_spkcnt: [n_{i} for i in range(nCells)]            
        """
        cell_spkcnt = n[cell, :][np.newaxis, :] # .shape: (1, nTimeBins)
        cell_ratemap = F[cell, :][:, np.newaxis] # .shape: (nFlatPositionBins, 1)
        coeff = 1.0 / (factorial(cell_spkcnt)) # 1/factorial(n_{i}) term # .shape: (1, nTimeBins)
        cum_cell_prob *= (((tau * cell_ratemap) ** cell_spkcnt) * coeff) * (np.exp(-tau * cell_ratemap)) # product equal using *=  # .shape (nFlatPositionBins, nTimeBins)

    posterior = cum_cell_prob # The product has already been accumulating all along
    posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n
    return posterior

def neuropy_bayesian_prob_full_only(tau, P_x, F, n):
    assert(len(n) == np.shape(F)[1]), f'n must be a column vector with an entry for each place cell (neuron). Instead it is of np.shape(n): {np.shape(n)}. np.shape(F): {np.shape(F)}'        
    nCells = n.shape[0]
    nTimeBins = n.shape[1] # many time_bins
    nFlatPositionBins = np.shape(P_x)[0]
    F = F.T # Transpose F so it's of the right form
    cell_prob = np.zeros((nFlatPositionBins, nTimeBins, nCells))
    for cell in range(nCells):
        """ Comparing to the Zhang paper: the output posterior is P_n_given_x (Eqn 35)
            cell_ratemap: [f_{i}(x) for i in range(nCells)]
            cell_spkcnt: [n_{i} for i in range(nCells)]            
        """
        cell_spkcnt = n[cell, :][np.newaxis, :]
        cell_ratemap = F[cell, :][:, np.newaxis]
        coeff = 1 / (factorial(cell_spkcnt)) # 1/factorial(n_{i}) term
        cell_prob[:, :, cell] = (((tau * cell_ratemap) ** cell_spkcnt) * coeff) * np.exp(-tau * cell_ratemap) # broadcasting
    posterior = np.prod(cell_prob, axis=2) # note this product removes axis=2 (nCells)
    posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n
    return posterior
    

In [ ]:
cell_spkcnt.shape # (1, 20760)

In [ ]:
cell_ratemap.shape # (63, 1)

In [ ]:
coeff.shape

In [ ]:
cell_prob.shape # (63, 20760) # (nFlatPositionBins, nTimeBins)

In [137]:
## save test data:
test_file = 'bayes_decoder_test_data.npz'
# np.savez(test_file, tau=tau, P_x=P_x, F=F, n=n)
## Load test data:
npzfile = np.load(test_file)
tau, P_x, F, n = npzfile['tau'], npzfile['P_x'], npzfile['F'].T, npzfile['n']

## "Flat" Implementation Exploration

In [115]:
tau, P_x, F, n = npzfile['tau'], npzfile['P_x'], npzfile['F'].T, npzfile['n']

nCells = n.shape[0]
nTimeBins = n.shape[1] # many time_bins
nFlatPositionBins = np.shape(P_x)[0]
F = F.T # Transpose F so it's of the right form
n = n.astype(F.dtype)

## Single-cell flat version which updates each iteration:
# cell_prob = np.zeros((nFlatPositionBins, nTimeBins)) ## MemoryError: Unable to allocate 65.4 GiB for an array with shape (3969, 21896, 101) and data type float64
# cell_probs_list = []

prev_cell_prob = np.ones((nFlatPositionBins, nTimeBins), dtype=F.dtype)
# cum_cell_probs_list = []

for cell in np.arange(nCells):
    """ Comparing to the Zhang paper: the output posterior is P_n_given_x (Eqn 35)
        cell_ratemap: [f_{i}(x) for i in range(nCells)]
        cell_spkcnt: [n_{i} for i in range(nCells)]            
    """
    
    # cell_spkcnt = n[cell, :] # .shape: (nTimeBins,)
    # cell_ratemap = F[cell, :] # .shape: (nFlatPositionBins,)
    cell_spkcnt = n[cell, :][np.newaxis, :] # .shape: (1, nTimeBins)
    cell_ratemap = F[cell, :][:, np.newaxis] # .shape: (nFlatPositionBins, 1)
    coeff = 1 / (factorial(cell_spkcnt)) # 1/factorial(n_{i}) term # .shape: (1, nTimeBins)
    cell_prob = (
        (
            (tau * cell_ratemap) ** cell_spkcnt
        ) * coeff
    ) * (
        np.exp(-tau * cell_ratemap)
    ) # .shape (nFlatPositionBins, nTimeBins)
    # cell_probs_list.append(cell_prob)
    
    prev_cell_prob = prev_cell_prob * cell_prob # multiply equals
    # cum_cell_probs_list.append(prev_cell_prob)
    
    # cell_prob *= (((tau * cell_ratemap) ** cell_spkcnt) * coeff) * (
    #     np.exp(-tau * cell_ratemap)
    # ) # product equal using *=

# cell_probs_list
posterior = prev_cell_prob # The product has already been accumulating all along
posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n
posterior ## all zeros

array([[2.16032911e-21, 1.07877452e-13, 9.99224320e-14, ...,
        1.08410046e-25, 4.29213290e-21, 5.06731636e-18],
       [8.59995291e-23, 5.76198969e-15, 4.92387777e-14, ...,
        8.46898179e-27, 2.94473854e-21, 5.45897691e-19],
       [7.84539082e-27, 1.18257634e-18, 3.54886252e-16, ...,
        8.07764100e-31, 8.00520321e-24, 1.62566404e-22],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [93]:
P_x.dtype # dtype('float64')
F.dtype # dtype('float64')
n.dtype # dtype('int64')
coeff.dtype # dtype('float64')
cell_prob.dtype # dtype('float64')

array([[4.60960624e-62, 3.54628562e-59, 1.27511999e-49, ...,
        4.55076146e-83, 6.46304760e-78, 2.36811517e-71],
       [1.83501655e-63, 1.89415496e-60, 6.28340889e-50, ...,
        3.55505025e-84, 4.43415565e-78, 2.55115037e-72],
       [1.67401172e-67, 3.88751623e-64, 4.52873839e-52, ...,
        3.39077594e-88, 1.20541490e-80, 7.59723572e-76],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [91]:
post_hoc_cell_prob = np.ones((nFlatPositionBins, nTimeBins))
for cell in np.arange(nCells):
    post_hoc_cell_prob = post_hoc_cell_prob * cell_probs_list[cell] # multiply equals
post_hoc_cell_prob

array([[4.60960624e-62, 3.54628562e-59, 1.27511999e-49, ...,
        4.55076146e-83, 6.46304760e-78, 2.36811517e-71],
       [1.83501655e-63, 1.89415496e-60, 6.28340889e-50, ...,
        3.55505025e-84, 4.43415565e-78, 2.55115037e-72],
       [1.67401172e-67, 3.88751623e-64, 4.52873839e-52, ...,
        3.39077594e-88, 1.20541490e-80, 7.59723572e-76],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [89]:
prev_cell_prob

array([[2.16032911e-21, 1.07877452e-13, 9.99224320e-14, ...,
        1.08410046e-25, 4.29213290e-21, 5.06731636e-18],
       [8.59995291e-23, 5.76198969e-15, 4.92387777e-14, ...,
        8.46898179e-27, 2.94473854e-21, 5.45897691e-19],
       [7.84539082e-27, 1.18257634e-18, 3.54886252e-16, ...,
        8.07764100e-31, 8.00520321e-24, 1.62566404e-22],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
prev_cell_prob

In [79]:
posterior = np.prod(cell_probs_list, axis=2) # The product has already been accumulating all along
posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n
posterior ## all zeros

C:\Users\pho\AppData\Local\Temp\ipykernel_22484\1457126432.py:2: RuntimeWarning: invalid value encountered in double_scalars
  posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n


nan

In [ ]:
cell_prob_3D = np.stack(cell_probs_list, axis=-1) # .shape: (nFlatPositionBins, nTimeBins, 105)
np.prod(cell_prob_3D, axis=2)

In [86]:
prev_cell_prob = np.ones((nFlatPositionBins, nTimeBins))
for a_cell_prob in cell_probs_list:
    prev_cell_prob = prev_cell_prob * a_cell_prob 
    
prev_cell_prob

array([[4.60960624e-62, 3.54628562e-59, 1.27511999e-49, ...,
        4.55076146e-83, 6.46304760e-78, 2.36811517e-71],
       [1.83501655e-63, 1.89415496e-60, 6.28340889e-50, ...,
        3.55505025e-84, 4.43415565e-78, 2.55115037e-72],
       [1.67401172e-67, 3.88751623e-64, 4.52873839e-52, ...,
        3.39077594e-88, 1.20541490e-80, 7.59723572e-76],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

(63, 20760, 105)

In [73]:
cum_cell_prob.shape

(63, 20760)

In [57]:
tau

In [58]:
(tau * cell_ratemap).shape # (nFlatPositionBins, 1)

In [59]:
((tau * cell_ratemap) ** cell_spkcnt).shape # (nFlatPositionBins, nTimeBins)

In [62]:
(((tau * cell_ratemap) ** cell_spkcnt) * coeff).shape # (nFlatPositionBins, nTimeBins)

In [60]:
np.exp(-tau * cell_ratemap).shape # (nFlatPositionBins, 1)

(63, 1)

(63, 20760)

## "Full" Implementation Exploration

In [101]:
## Full Implementation:
tau, P_x, F, n = npzfile['tau'], npzfile['P_x'], npzfile['F'].T, npzfile['n']
cell_prob = None
nCells = n.shape[0]
nTimeBins = n.shape[1] # many time_bins
nFlatPositionBins = np.shape(P_x)[0]
F = F.T # Transpose F so it's of the right form
cell_prob = np.zeros((nFlatPositionBins, nTimeBins, nCells))
for cell in range(nCells):
    """ Comparing to the Zhang paper: the output posterior is P_n_given_x (Eqn 35)
        cell_ratemap: [f_{i}(x) for i in range(nCells)]
        cell_spkcnt: [n_{i} for i in range(nCells)]            
    """
    cell_spkcnt = n[cell, :][np.newaxis, :]
    cell_ratemap = F[cell, :][:, np.newaxis]
    coeff = 1 / (factorial(cell_spkcnt)) # 1/factorial(n_{i}) term
    cell_prob[:, :, cell] = (((tau * cell_ratemap) ** cell_spkcnt) * coeff) * np.exp(-tau * cell_ratemap) # broadcasting
    
posterior = np.prod(cell_prob, axis=2) # note this product removes axis=2 (nCells)
posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n

In [106]:
P_x.dtype # dtype('float64')
F.dtype # dtype('float64')
n.dtype # dtype('int64')
coeff.dtype # dtype('float64')
cell_prob.dtype # dtype('float64')

dtype('float64')

In [70]:
i = 2
(cell_prob[:,:,i] == cell_probs_list[i]).all()

In [83]:
non_normalized_posterior = np.prod(cell_prob, axis=2) # note this product removes axis=2 (nCells)
non_normalized_posterior

array([[4.60960624e-62, 3.54628562e-59, 1.27511999e-49, ...,
        4.55076146e-83, 6.46304760e-78, 2.36811517e-71],
       [1.83501655e-63, 1.89415496e-60, 6.28340889e-50, ...,
        3.55505025e-84, 4.43415565e-78, 2.55115037e-72],
       [1.67401172e-67, 3.88751623e-64, 4.52873839e-52, ...,
        3.39077594e-88, 1.20541490e-80, 7.59723572e-76],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [82]:
non_normalized_posterior = np.prod(cell_prob_3D, axis=2) # note this product removes axis=2 (nCells)
non_normalized_posterior

array([[4.60960624e-62, 3.54628562e-59, 1.27511999e-49, ...,
        4.55076146e-83, 6.46304760e-78, 2.36811517e-71],
       [1.83501655e-63, 1.89415496e-60, 6.28340889e-50, ...,
        3.55505025e-84, 4.43415565e-78, 2.55115037e-72],
       [1.67401172e-67, 3.88751623e-64, 4.52873839e-52, ...,
        3.39077594e-88, 1.20541490e-80, 7.59723572e-76],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [78]:
(cell_prob_3D == cell_prob).all()

True

In [ ]:
posterior

In [47]:
posterior.shape # (63, 20760)

(63, 20760)

In [ ]:
cell_probs_list

In [20]:
np.nonzero(posterior)

In [25]:
cell_spkcnt.shape # (1, 20760)
cell_ratemap.shape # (63, 1)
coeff.shape # (1, 20760)

(1, 20760)

## Versions

In [144]:
tau, P_x, F, n = npzfile['tau'], npzfile['P_x'], npzfile['F'].T, npzfile['n']

In [146]:
posterior = neuropy_bayesian_prob(tau, P_x, F, n, use_flat_computation_mode=True)
posterior

posterior_full_only = neuropy_bayesian_prob(tau, P_x, F, n, use_flat_computation_mode=False)


# C:\Users\pho\AppData\Local\Temp\ipykernel_22484\1973842464.py:68: RuntimeWarning: invalid value encountered in true_divide
#   posterior /= np.sum(posterior, axis=0) # C(tau, n) = np.sum(posterior, axis=0): normalization condition mentioned in eqn 36 to convert to P_x_given_n

In [150]:
assert (posterior_full_only == posterior).all()

In [145]:
p

array([[2.16032911e-21, 1.07877452e-13, 9.99224320e-14, ...,
        1.08410046e-25, 4.29213290e-21, 5.06731636e-18],
       [8.59995291e-23, 5.76198969e-15, 4.92387777e-14, ...,
        8.46898179e-27, 2.94473854e-21, 5.45897691e-19],
       [7.84539082e-27, 1.18257634e-18, 3.54886252e-16, ...,
        8.07764100e-31, 8.00520321e-24, 1.62566404e-22],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [125]:
posterior_full_only = neuropy_bayesian_prob_full_only(tau, P_x, F, n)
posterior_full_only

array([[2.16032911e-21, 1.07877452e-13, 9.99224320e-14, ...,
        1.08410046e-25, 4.29213290e-21, 5.06731636e-18],
       [8.59995291e-23, 5.76198969e-15, 4.92387777e-14, ...,
        8.46898179e-27, 2.94473854e-21, 5.45897691e-19],
       [7.84539082e-27, 1.18257634e-18, 3.54886252e-16, ...,
        8.07764100e-31, 8.00520321e-24, 1.62566404e-22],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [130]:
posterior_flat_only = neuropy_bayesian_prob_flat_only(tau, P_x, F, n)
posterior_flat_only

array([[2.16032911e-21, 1.07877452e-13, 9.99224320e-14, ...,
        1.08410046e-25, 4.29213290e-21, 5.06731636e-18],
       [8.59995291e-23, 5.76198969e-15, 4.92387777e-14, ...,
        8.46898179e-27, 2.94473854e-21, 5.45897691e-19],
       [7.84539082e-27, 1.18257634e-18, 3.54886252e-16, ...,
        8.07764100e-31, 8.00520321e-24, 1.62566404e-22],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [100]:
posterior_fixed = neuropy_bayesian_prob_fixed(tau, P_x, F, n)
posterior_fixed

## Finally, test ZhangReconstructionImplementation implementation

In [152]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import ZhangReconstructionImplementation

C:\Users\pho\miniconda3\envs\phoviz_ultimate\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [153]:
tau, P_x, F, n = npzfile['tau'], npzfile['P_x'], npzfile['F'].T, npzfile['n']

In [154]:
posterior = neuropy_bayesian_prob(tau, P_x, F, n, use_flat_computation_mode=True)
posterior_full_only = neuropy_bayesian_prob(tau, P_x, F, n, use_flat_computation_mode=False)

In [155]:
assert (posterior_full_only == posterior).all()